# Ch10 機器視覺實戰演練：CNN (Convolutional Neural Network)

#### 10.3.1 匯入模組

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D # new!
                           # 將多軸轉成一軸   卷積層   最大池化層 

#### 10.3.2 載入資料集並做資料預處理

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()  

訓練資料要傳遞給Conv2D()時，必須先轉成4軸陣列的形式，分別是(樣本數 * 寬 * 高 * 顏色通道)

In [16]:
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32')  #(60000張手寫字,寬=28,高=28,由於MNIST數字為單色影像，所以設為1，若全採則設為3)
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32')    # astype() 會將像素質從整數轉為浮點數

縮小X範圍

In [18]:
X_train /= 255    
X_test /= 255

將標籤y轉換為one-hot encoding

In [13]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

#### 10.3.3 規劃 CNN 模型的架構

In [22]:
model = Sequential()

#第一層隱藏層
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
#    卷積層的濾鏡數=32個  濾鏡尺寸=3*3的眷積核   激活函數用Relu  輸入影像為28*28會得到26*26的特徵圖、步長為1 

#第二層隱藏層
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))  #濾鏡數為64
model.add(MaxPooling2D(pool_size=(2, 2)))    #加入 最大池化層 用來繼續減少特徵圖的尺寸 滑動窗口尺寸=2*2，步長預設為2
model.add(Dropout(0.25))   #降低模型overfitting
model.add(Flatten())   #會把MaxPooling輸出的特徵圖展平為1D陣列，這樣才能把這些像素值傳給"密集層"(只接受1D作為輸入)

#搭配丟棄法的密集隱藏層
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#輸出層
model.add(Dense(n_classes, activation='softmax'))   #利用softmax換成各類別的機率

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 9216)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1179776   
                                                      

#### 10.3.4 編譯、訓練模型

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
#註：由於神經網路的初始權重參數是隨機設定的, 參雜了隨機性, 因此底下 (或您重跑一次) 的結果不會與書中完全一樣, 但模型的能力是相近的
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
469/469 [==============================] - 46s 97ms/step - loss: 1.0465 - accuracy: 0.6525 - val_loss: 0.3234 - val_accuracy: 0.9057
Epoch 2/10
469/469 [==============================] - 46s 97ms/step - loss: 0.4686 - accuracy: 0.8562 - val_loss: 0.2414 - val_accuracy: 0.9276
Epoch 3/10
226/469 [=============>................] - ETA: 22s - loss: 0.4209 - accuracy: 0.8715

KeyboardInterrupt: 